In [1]:
import requests

## Play-by-Play Data

`GET https://statsapi.web.nhl.com/api/v1/game/ID/feed/live Returns` all data about a specified game id including play data with on-ice coordinates and post-game details like first, second and third stars and any details about shootouts. The data returned is simply too large at often over 30k lines and is best explored with a JSON viewer.

### Game ID Dictionary
* First 4 digits signify the season start year, ex: `2018` (for the 2018-2019 season)
* Next 2 digits signify the following:
    - 01: Preseason
    - 02: Regular Season
    - 03: Post-Season (Playoffs)
    - 04: All-Star Games
* The final 4 digits signify the game number. Valid range is `0001`-`1271` (until 2020, when the NHL will mandate that there will be 1312 games per season)

Note: for help visualizing JSON see - http://jsonviewer.stack.hu

In [2]:
year = '2018'
season = '02'
game_number = '0683'
game_id = year + season + game_number

url = f'https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live'
response = requests.get(url)

In [3]:
url

'https://statsapi.web.nhl.com/api/v1/game/2018020683/feed/live'

In [4]:
json = response.json()

In [5]:
plays = json.get('liveData').get('plays').get('allPlays')

In [7]:
faceoffs = [play.get('result') for play in plays if play.get('result').get('event') == 'Faceoff']

In [17]:
events = [play.get('result').get('event') for play in plays]
event_ids = [play.get('about').get('eventId') for play in plays]
periods = [play.get('about').get('period') for play in plays]

unique_events = list(set(events))
unique_event_ids = list(set(event_ids))

In [25]:
event_dict = {}

for event, eid, period in zip(events, event_ids, periods):
    if event in event_dict:
        # key exists, add event id and period to the dict
        event_dict[event]['eventId'].append(eid)
        event_dict[event]['period'].append(period)
    else:
        # key does not exist, create the nested dict
        event_dict[event] = {'eventId' : [],
                             'period' : []}

In [26]:
event_dict

{'Blocked Shot': {'eventId': [19,
   24,
   28,
   30,
   45,
   48,
   49,
   312,
   325,
   329,
   331,
   335,
   341,
   349,
   502,
   505],
  'period': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]},
 'Faceoff': {'eventId': [66,
   71,
   25,
   29,
   84,
   88,
   91,
   34,
   98,
   201,
   211,
   42,
   43,
   47,
   218,
   221,
   222,
   226,
   234,
   314,
   317,
   240,
   243,
   244,
   247,
   328,
   334,
   336,
   337,
   339,
   413,
   415,
   347,
   422,
   503],
  'period': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2]},
 'Game Scheduled': {'eventId': [], 'period': []},
 'Giveaway': {'eventId': [57,
   59,
   63,
   89,
   99,
   206,
   214,
   228,
   229,
   238,
   239,
   250,
   408,
   409,
   419,
   420,
   426],
  'period': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]},
 'Go

In [18]:
event_dict = {}

for event, eid in zip(events, event_ids):
    if event in event_dict:
        # key exsists, append to list
        event_dict[event].append(eid)
    else:
        # key does not exists, create an empty list
        event_dict[event] = []

In [19]:
event_dict

{'Blocked Shot': [19,
  24,
  28,
  30,
  45,
  48,
  49,
  312,
  325,
  329,
  331,
  335,
  341,
  349,
  502,
  505],
 'Faceoff': [66,
  71,
  25,
  29,
  84,
  88,
  91,
  34,
  98,
  201,
  211,
  42,
  43,
  47,
  218,
  221,
  222,
  226,
  234,
  314,
  317,
  240,
  243,
  244,
  247,
  328,
  334,
  336,
  337,
  339,
  413,
  415,
  347,
  422,
  503],
 'Game Scheduled': [],
 'Giveaway': [57,
  59,
  63,
  89,
  99,
  206,
  214,
  228,
  229,
  238,
  239,
  250,
  408,
  409,
  419,
  420,
  426],
 'Goal': [303, 320, 321, 324, 346],
 'Hit': [58,
  60,
  61,
  62,
  64,
  67,
  68,
  69,
  72,
  75,
  76,
  79,
  80,
  85,
  92,
  94,
  95,
  202,
  205,
  216,
  223,
  224,
  227,
  230,
  242,
  249,
  402,
  404,
  406,
  410,
  411,
  414,
  417,
  418],
 'Missed Shot': [10,
  11,
  17,
  18,
  21,
  31,
  37,
  38,
  44,
  311,
  315,
  316,
  323,
  330,
  342,
  345,
  348,
  350,
  504],
 'Penalty': [219, 232],
 'Period End': [],
 'Period Official': [],
 'Period Re

In [9]:
[(play.get('about').get('eventIdx'), 
  play.get('about').get('eventId'), 
  play.get('result').get('event')) for play in plays]

[(0, 1, 'Game Scheduled'),
 (1, 4, 'Period Ready'),
 (2, 52, 'Period Start'),
 (3, 53, 'Faceoff'),
 (4, 54, 'Giveaway'),
 (5, 55, 'Takeaway'),
 (6, 56, 'Hit'),
 (7, 57, 'Giveaway'),
 (8, 7, 'Shot'),
 (9, 12, 'Shot'),
 (10, 14, 'Missed Shot'),
 (11, 8, 'Shot'),
 (12, 58, 'Hit'),
 (13, 9, 'Shot'),
 (14, 59, 'Giveaway'),
 (15, 60, 'Hit'),
 (16, 61, 'Hit'),
 (17, 62, 'Hit'),
 (18, 10, 'Missed Shot'),
 (19, 11, 'Missed Shot'),
 (20, 63, 'Giveaway'),
 (21, 64, 'Hit'),
 (22, 65, 'Stoppage'),
 (23, 66, 'Faceoff'),
 (24, 67, 'Hit'),
 (25, 68, 'Hit'),
 (26, 69, 'Hit'),
 (27, 13, 'Shot'),
 (28, 70, 'Stoppage'),
 (29, 71, 'Faceoff'),
 (30, 15, 'Blocked Shot'),
 (31, 16, 'Shot'),
 (32, 17, 'Missed Shot'),
 (33, 72, 'Hit'),
 (34, 18, 'Missed Shot'),
 (35, 21, 'Missed Shot'),
 (36, 19, 'Blocked Shot'),
 (37, 73, 'Takeaway'),
 (38, 74, 'Takeaway'),
 (39, 20, 'Shot'),
 (40, 75, 'Hit'),
 (41, 22, 'Shot'),
 (42, 76, 'Hit'),
 (43, 23, 'Shot'),
 (44, 24, 'Blocked Shot'),
 (45, 77, 'Stoppage'),
 (46, 25, 'F

In [10]:
[ (play.get('players')[0].get('player').get('fullName'),
   play.get('players')[1].get('player').get('fullName') )
 for play in plays
 if play.get('result').get('event') == 'Faceoff' ]

[('Jack Eichel', 'Sebastian Aho'),
 ('Vladimir Sobotka', 'Lucas Wallmark'),
 ('Greg McKegg', 'Evan Rodrigues'),
 ('Lucas Wallmark', 'Kyle Okposo'),
 ('Victor Rask', 'Johan Larsson'),
 ('Evan Rodrigues', 'Victor Rask'),
 ('Jack Eichel', 'Sebastian Aho'),
 ('Sebastian Aho', 'Jack Eichel'),
 ('Lucas Wallmark', 'Vladimir Sobotka'),
 ('Johan Larsson', 'Justin Williams'),
 ('Johan Larsson', 'Victor Rask'),
 ('Vladimir Sobotka', 'Greg McKegg'),
 ('Vladimir Sobotka', 'Greg McKegg'),
 ('Greg McKegg', 'Sam Reinhart'),
 ('Lucas Wallmark', 'Jeff Skinner'),
 ('Victor Rask', 'Jack Eichel'),
 ('Jeff Skinner', 'Lucas Wallmark'),
 ('Johan Larsson', 'Greg McKegg'),
 ('Jack Eichel', 'Sebastian Aho'),
 ('Jack Eichel', 'Sebastian Aho'),
 ('Lucas Wallmark', 'Jeff Skinner'),
 ('Kyle Okposo', 'Sebastian Aho'),
 ('Johan Larsson', 'Victor Rask'),
 ('Jack Eichel', 'Sebastian Aho'),
 ('Vladimir Sobotka', 'Greg McKegg'),
 ('Johan Larsson', 'Lucas Wallmark'),
 ('Lucas Wallmark', 'Johan Larsson'),
 ('Sebastian Aho',